In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab5-deconvolution.ipynb")

# EE 120 Lab 5: Deconvolution and Imaging

**Signals and Systems** at UC Berkeley

Acknowledgements:

- **Spring 2019** (v1.0): Dominic Carrano, Ilya Chugunov, Babak Ayazifar  
- **Fall 2019** (v2.0): Dominic Carrano, Ilya Chugunov  
- **Spring 2020** (v2.1): Dominic Carrano, Ilya Chugunov
- **Spring 2022** (v3.0): Anmol Parande

In [ ]:
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
from scipy import signal, linalg
from scipy.io import wavfile
%matplotlib inline

# Background

Note: For simplicity, we'll consider all signals and systems in this lab as discrete-time entities, since that's what computers use. In truth, there are other steps involved, such as sampling and quantization.

### Motivation: Signal Restoration

*Degradation* (also referred to as *distortion*) is a common problem in signal and image processing: you want access to a "ground truth" signal $x(n)$, but due to some real-world physical constraints, you can only observe $y(n)$, a corrupted version of $x(n)$. 

For example, suppose you're playing a song $x(n)$ in a large concert hall. Due to the acoustics of the room, your audience hears $y(n)$, the song superimposed with a quieter echo of it (that is, $x(n)$ plus a delayed and attenuated copy of $x(n)$).

In this lab we address how to compensate for signal distortions. Our goal is to produce $\hat{x}(n)$, an estimate of the original signal $x(n)$, using the measured signal $y(n)$. The process of determining $\hat{x}(n)$ using $y(n)$ is called *signal restoration*.

### Problem Formulation

From a block-diagram perspective, we can think of this *degradation* as a *system* $H$, which outputs $y(n)$, our measured signal, from input $x(n)$, the true signal. 

<img src="figs/degrade.png" width="700px"></img>

Our job is to design a *restoration system*, $R$, that attempts to undo the effects of $H$ to give us $\hat{x}(n)$, a nice approximation of $x(n)$. Using the concert hall example from above, $H$ could be the acoustics of the room, and $R$ could be a specialized filter chip in a de-echoing concert microphone you bought from your local microphone dealer.

### Designing $R$

In general, $H$ could be *any* system (e.g., nonlinear, time-varying, etc.), and undoing it may be intractable. To simplify things, we'll assume: 
- $H$, the degradation system,
is DT-LTI (with frequency response $H(\omega)$ and impulse response $h(n)$).
- $R$, the recovery system we choose, is DT-LTI (with frequency response $R(\omega)$ and impulse response $r(n)$).

Although it's not obvious, in many scenarios of practical interest (e.g. acoustic echoes, system lag, image blur), this simplified model where we force ourselves to think of everything as LTI is not too far from the real-life system behaviour.

Since $H$ is LTI, we can describe it's behaviour as a convolution, $y(n) = (x * h)(n)$. We want to design $R$ to *undo* this convolution, performing what's called ***deconvolution***. For this problem it's easier to work in the frequency domain, where our time-signal convolution just becomes frequency-response multiplication:

$$Y(\omega) = H(\omega) X(\omega),$$

and

$$\hat{X}(\omega) = R(\omega) Y(\omega) = R(\omega) H(\omega) X(\omega),$$

Algebraically, if we pick $R(\omega) = 1 / H(\omega)$, this will give us $\hat{X}(\omega) = X(\omega)$, from which we can take the inverse DTFT to recover the original signal. Effectively, we compute $\hat{X}(\omega) = Y(\omega) / H(\omega)$.

This algorithm is known as *Fourier deconvolution* (sometimes also called "inverse filtering" or "direct deconvolution"), since we perform the deconvolution directly based on the Fourier Transform(s) of the systems involved.

### Issues and Alternatives

Fourier deconvolution computes the multiplicative inverse of $H$'s frequency response, and uses it to characterize the inverse system $R$. 

This approach has two main issues:
1. $H$ may be zero at some frequency $\omega_0$, in which case the division is not well-defined. Intuitively, all content at $\omega_0$ is killed, and we're left with no information about it, so we can't invert the behavior.
2. Even if $H$ is nonzero over all frequencies, it's often very small, and so $1 / H(\omega)$ will be huge, and amplify noise that will be present in practical setups.

Due to these issues, it's more common to use *Wiener filtering* in practice for deconvolution. Wiener filtering is a more sophisticated technique that uses statistical properties of the signals and noise involved to produce better results. However, Wiener filtering and Fourier deconvolution are similar in spirit (and we don't want to go down a rabbit hole into statistical signal processing), so we'll be using Fourier in this lab.

Later in the lab, we'll explore these issues and how they can affect our signals.

# Q1: Echo Cancellation

A classic problem in signals and systems, acoustics, and digital communications is *echo cancellation*. A sender transmits a signal to someone, and they receive it, along with a delayed, attenuated copy of it. 

There are many causes of this phenomenon which you can read about in the references, including:
- Signal back reflections due to impedance mismatches in electronic circuits (see references 1, 2).
- Audio feedback in microphones (see reference 3).
- Acoustic properties of the space the signal is being transmitted in. For example, if you send a signal indoors, it may go in multiple directions, with part of the signal going straight to a receiver, and the rest of it bouncing off of several walls before it arrives as a delayed and attenuated copy of the first.

## Modelling an echo

Many different models for echoes have been considered, and you can find a more comprehensive treatment of this subject in references 4 and 5. Here, we'll consider one of the simpler models for an echo, where our communication channel (the degradation system) is an LTI system with impulse response

$$h(n) = \delta(n) + \alpha \delta(n - k)$$

where $0 < \alpha < 1$ is the attenuation of the echo and $k > 0$ is the integer delay of the echo. Assuming $0 < \alpha < 1$ means the copy is weaker than the original, and $k>0$ that the copy shows up after the original.

We can think of this as a channel that transmits the signal perfectly and instantaneously, and also with a $k$-step delay and some attenuation along an echo path. In this problem, we'll send some audio over this channel, and try to undo the corruptions it introduces using deconvolution.

## A Brief Intro to Audio Signals

In this question, we'll use ["Take on Me" by a-ha](https://www.youtube.com/watch?v=djV11Xbc914) as our test signal. We have provided it for you as a [.wav](https://en.wikipedia.org/wiki/WAV) (pronounced "wave") file. Run the cell below and have a listen! It's normal if the cell takes a few seconds to load. You'll see a graphic display pop up with a play button once it's finished loading.

In [ ]:
ipd.Audio("wavs/TakeOnMe.wav")

Now, we'll read the file as a numpy array using scipy's WAV file API. The `read` function returns two things: the sampling rate of the audio, and the signal itself.

In [ ]:
# In signal processing code, "fs" is conventionally used for sampling frequency in Hertz (cycles/second)
fs, data = wavfile.read("wavs/TakeOnMe.wav")

Typically, digital audio is sampled at 44.1 kHz, or 44100 Hz, although some more modern formats use 48 kHz. This is motivated by the fact that the human ear can only hear up to ~20 kHz. Given this, the Nyquist criterion suggests that the sampling rate should be at least 40 kHz, so some extra wiggle room is added on. We can easily verify that we're dealing with a sampling rate of 44.1 kHz.

In [ ]:
print(fs) # sampling rate in Hz

When dealing with real world data, a good first step before processing it is checking what size it is using `np.shape`, just as we did when building our heart rate detector in Lab 4.

In [ ]:
np.shape(data)

Perhaps surprisingly, the song, when read in as a signal, is actually a 1984500-by-2 matrix! Why?

The track's runtime is 45 seconds, and with a sampling rate of 44.1 kHz, we expect a total of 

$$45\ \text{seconds} \cdot \frac{44100\ \text{samples}}{1\ \text{second}} = 1984500\ \text{samples}$$

in our data. That explains the first dimension. Why a two column matrix, though?

The reason we have two separate columns of data, rather than a single array of 1984500 samples, is due to the use of [two-channel audio](https://en.wikipedia.org/wiki/Stereophonic_sound). When you listen to music with a pair of headphones, each ear is receiving a separate audio stream, hence the need for two samples at each point in time. The same principle applies to laptops or other sound systems with two speakers. 

**What this means for us is that each channel (i.e., column of this matrix) should be processed as a distinct, 1D signal.**

As a final step before moving on, we'll crop our song to the first 10 seconds. This will make processing go much faster, and we'll still be able to hear what's going on throughout the process.

In [ ]:
n_sec = 10
data_cropped = data[:n_sec * fs, :]

As a sanity check, run the cell below to play the first 10 seconds of the song. The display should show that the file has 10 seconds of audio, and it should sound exactly the same as before.

In [ ]:
wavfile.write("wavs/cropped_TakeOnMe.wav", fs, data_cropped)
ipd.Audio("wavs/cropped_TakeOnMe.wav")

Now let's make some echoes!

## Q1a: Transmission

Implement the function `transmit` below to simulate the echo channel. As a reminder, so that you don't have to keep scrolling back and forth, we're modelling the channel as an LTI system with impulse response

$$h(n) = \delta(n) + \alpha \delta(n-k)$$

where $\alpha$ (the attentuation factor) and $k$ (the delay of the echo in samples) are provided to you as function arguments, along with the signal to transmit, $x$. Your function should return the result of transmitting $x$ over the channel, performing the parasitic convolution we'll later be trying to undo. 

**As a reminder, each audio channel should be considered as a distinct signal requiring a separate convolution when transmitting.**

#### Quantization

All audio we're working with is [quantized](https://en.wikipedia.org/wiki/Quantization) to 16 bits. After processing our signal, we have to renormalize each entry to be a 16-bit integer value, or we'll introduce new distortions to it.

After you transmit the song clip **and reassemble it into a two-channel matrix**, say `x_echoed`, apply the following line of code: 

**`np.int16(x_echoed / np.max(np.abs(x_echoed)) * np.iinfo(np.int16).max)`**. 

This fits every value to the range [-1, +1] and then rescales it to be within the range of [-32767, 32767]. This will be the last thing you have to do before returning the result.

**Hint:** All convolutions should be done in "full" mode to avoid cutting out data. Since we're using "full", there's no need to pad implicit zeros onto the echo channel impulse response; it should only be length $k+1$.

In [ ]:
def transmit(x, alpha, k):
    """
    Simulate transmission of a two-channel audio signal x over an LTI echo channel which sends 
    x and a copy of x delayed by k > 0 samples and attenuated by a factor 0 < alpha < 1.
    
    Parameters:
    x        - The audio signal. Assumed to be an Nx2 matrix, where N is the number of audio samples.
    alpha    - The attenuation factor. Assumed that 0 < alpha < 1.
    k        - The delay factor, in samples. Assumed that k > 0.
    
    Returns:
    x_echoed - The echoed signal.
    """
    ...

In [ ]:
grader.check("q1a")

Once you've finished implementing `transmit`, try it out by running the cell below, which will generate an echo at $80\%$ the strength of the original signal and with a delay of $2 \cdot f_s = 88200$ samples (exactly two seconds). 

**This means our transmitted song will be 12 seconds long.** The original copy starts at time zero and finishes 10 seconds in. The echoed copy starts 2 seconds in, and ends after 12 seconds from the start of the original copy.

This cell will take anywhere from several seconds to a minute to run depending on your laptop. Even with 10 seconds of data, we have two $10 \cdot 44100$ entry convolutions to compute, which will take some time.

In [ ]:
x_echo = transmit(data_cropped, .8, 2*fs)

Run the cell below to play your echo-corrupted song. 

You should hear a second copy of the track that comes in two seconds later. This means that the first and last two seconds of the audio should only contain one track. The first two seconds will contain the start of the original, and the last two seconds will be the end of the echo. It should be easy to tell if your result is correct or not by just listening.

In [ ]:
wavfile.write("wavs/echoed_TakeOnMe.wav", fs, x_echo)
ipd.Audio("wavs/echoed_TakeOnMe.wav")

## Q1b: Deconvolving

Implement the function `deconvolve` below using the Fourier deconvolution algorithm described in the background. Feel free to use NumPy's [fft](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fft.html#numpy.fft.fft) and [inverse fft](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.ifft.html#numpy.fft.ifft) functions for this part.

As with Lab 3's question on oscilloscope signal alignment, we're going to encounter the issue of numerical noise here yielding an erroneously nonzero imaginary part in our final result. **Be sure to take the real part of any signal you return to avoid a fake imaginary part showing up.**

In [ ]:
def deconvolve(y, h):
    """
    Perform a Fourier deconvolution to deconvolve h "out of" y, assuming
    that h, y and the deconvolved signal are both real-valued.
    
    Parameters:
    y - The signal to deconvolve h out of.
    h - The impulse response used in the parasitic convolution.
    """
    ...

As a sanity check, let's do a toy example of echo cancellation. 

We'll set:
- $x(n) = .5 \delta(n) + \delta(n-1) + .5 \delta(n-2)$, a three-point pulse.
- $h(n) = \delta(n) + .4 \delta(n - 7)$, which will generate an echo with a seven-sample delay at $40\%$ the original's strength.

Run the cell below to see what this looks like.

In [ ]:
x = np.array([.5, 1, .5]); h = np.array([1, 0, 0, 0, 0, 0, 0, .4]); y = np.convolve(x, h, "full")

x_pad = np.concatenate((x, np.zeros(len(y) - len(x))))
h_pad = np.concatenate((h, np.zeros(len(y) - len(h))))

plt.figure(figsize=(16, 4))
plt.subplot(1, 3, 1); plt.stem(x_pad); plt.title("Original signal x")
plt.subplot(1, 3, 2); plt.stem(h_pad, linefmt="C1", markerfmt="C1o"); plt.title("Impulse response h")
plt.subplot(1, 3, 3); plt.stem(y, linefmt="C2", markerfmt="C2o"); plt.title("Echoed version y = h * x")
plt.show()

We want to remove that second pulse that shows up in $y$. 

This example is indeed "toy" --- the echoed pulse and the original don't temporally overlap at all, so we could just zero out the echo to solve the problem. In the real setup, however, $k$ will be small compared to the signal length and the echo and original will be superimposed. In that case, if we zeroed out the samples, we'd be cutting out data, too, and our song might sound bad or just be missing a few seconds of music altogether. Still, this is a great example to test with, since it'll be obvious if we succeed in killing the echo.

Run the cell below, and see if you pass the sanity check. The plots for $x$ and $\hat{x}$ should be identical (minus trivial differences like color).

In [ ]:
x_hat = deconvolve(y, h_pad)

plt.figure(figsize=(16, 4))

plt.subplot(1, 3, 1); plt.stem(x_pad); plt.title("Original signal x")
plt.subplot(1, 3, 2); plt.stem(y, linefmt="C2", markerfmt="C2o"); plt.title("Echoed version y")
plt.subplot(1, 3, 3); plt.stem(x_hat, linefmt="C4", markerfmt="C4o"); plt.title("Recovered version x_hat")

plt.show()

Here some more test cases for sanity checks.

In [ ]:
grader.check("q1b")

## Q1c: Echo Removal

Implement `cancel_echo` below, which removes an echo of strength $\alpha$ and sample delay $k$ from the signal `x_echo`. We want `cancel_echo(transmit(x, alpha, k), alpha, k)` to return `x` (possibly with extra zeros on the end, which are harmless) for any valid choices of $\alpha, k$. 

Don't forget that:
1. We must again renormalize the final output audio matrix to 16-bit integer values the way we did in `transmit`.
2. The two audio channels must be treated as separate 1D signals. 

**Hint:** In `deconvolve`, the FFT vectors we divide must be the same length. This means that unlike in `transmit`, where you only defined the impulse response over $k+1$ indices, you should zero pad it to the same length as `x_echo` before doing the deconvolutions. An example of this is in the Q1b sanity check.

In [ ]:
def cancel_echo(x_echo, alpha, k):
    """
    Cancel an alpha-strength, k-sample delay echo from a two-channel audio signal x_echo
    where k > 0 and 0 < alpha < 1.
    
    Parameters:
    x_echo     - The echo-corrupted audio signal. Assumed to be an Nx2 matrix, where N is the number of audio samples.
    alpha      - The attenuation factor. Assumed that 0 < alpha < 1.
    k          - The delay factor, in samples. Assumed that k > 0.
    
    Returns:
    x_echoless - The signal with the echo cancelled.
    """
    ...

Now, run the cell below to see how well your echo cancellation algorithm works! If it's correct:
- The audio file will be 12 seconds long.
- The first 10 seconds will be the original copy of the song.
- The last 2 seconds will be empty, as those audio samples are all zeros. Since we cancelled the echo, which was the only thing present at the end of our echoed recording, there's now no music there. Don't worry about these data-less samples, they're harmless. We could crop them to get the exact same signal if we really wanted to, but it doesn't matter as we don't hear anything.

In [ ]:
x_cleaned = cancel_echo(x_echo, .8, 2*fs)
wavfile.write("wavs/echoless.wav", fs, x_cleaned)
ipd.Audio("wavs/echoless.wav")

Once you've successfully removed the echo, move on to Q1d.

## Q1d: Noisy Deconvolution

We removed an echo from a *clean* audio recording, which is no small feat! However, we haven't accounted for noise—we've assumed that the only unwanted effect that occurs is due to the echo itself. Fourier deconvolution's main flaw is that it tends to amplify noise, a problem which we'll now explore.

### Noise Model

We'll assume an *additive* noise model: after the parasitic convolution, there is a *noise signal*, which we'll denote as $z$, that is added to the final signal just before measurement. Below is a block diagram. Note that unlike $x$ and $h$, $z$ is random, although we won't dive too much into that aspect.

<img src="figs/deconv_noise_model.png" alt="drawing" style="width:500px;"/>

Now, we assume $y(n) = (x * h)(n) + z(n)$, and we again want to recover $x$ given only $y, h$. To get a sense for how a noised, echoed signal differs from a noiseless one, let's add a small amount of white Gaussian noise to `x_echo`, the signal from before.

In [ ]:
x_echo_noisy = x_echo + 800 * np.random.randn(x_echo.shape[0], x_echo.shape[1])
x_echo_noisy = np.int16(x_echo_noisy / np.max(np.abs(x_echo_noisy)) * 32767)

wavfile.write("wavs/echoed_noised_TakeOnMe.wav", fs, x_echo_noisy)
ipd.Audio("wavs/echoed_noised_TakeOnMe.wav")

Now, we'll call the same echo cancellation function from before, and see what happens.

In [ ]:
x_cleaned_noisy = cancel_echo(x_echo_noisy, .8, 2*fs)
wavfile.write("wavs/echoless_noisy.wav", fs, x_cleaned_noisy)
ipd.Audio("wavs/echoless_noisy.wav")

<!-- BEGIN QUESTION -->

**Q:** How does the noised, echoed version sound (the one we get by running the first cell in this part) compared to the echoed version from before? You should still hear the echo, but there's also white noise that's been added. What does the white noise sound like?

**Q:** You should hear that the echo was removed, just as in the noiseless case. What about the noise, though? Is it louder or softer than before the deconvolution?

<span style="color:blue">**A:** (TODO) </span>
    
<span style="color:blue">**A:** (TODO) </span>

<!-- END QUESTION -->

### Analyzing Noisy Deconvolution

Just as understanding how to deconvolve signals is most easily done in the frequency domain, so is performing an error analysis. We know that 

$$y(n) = (x * h)(n) + z(n) \implies Y(\omega) = X(\omega)H(\omega) + Z(\omega).$$

The Fourier deconvolution algorithm returns $\hat{X}(\omega)$, an estimate of $X(\omega)$, which is computed as 

$$\hat{X}(\omega) = \dfrac{Y(\omega)}{H(\omega)} = \dfrac{X(\omega)H(\omega) + Z(\omega)}{H(\omega)} = X(\omega) + \dfrac{Z(\omega)}{H(\omega)},$$

so the difference between our estimate, $\hat{X}$, and the true spectrum, $X$, is

$$\hat{X}(\omega) - X(\omega) = \dfrac{Z(\omega)}{H(\omega)}.$$

In general, this will be a complex number, which isn't very useful as a way to quantify error. Instead, we can consider the *magnitude* of the error between the estimated spectrum and the true one:

$$|\hat{X}(\omega) - X(\omega)| = \dfrac{|Z(\omega)|}{|H(\omega)|}$$.

As a final sanity check on your understanding of how Fourier deconvolution performs in the presence of noise, answer the following questions. 

Assume that for all $\omega$, $|Z(\omega)| = \sigma$, which roughly says that the noise is "equally strong" at all frequencies. Truly, $z$ is random, so it's wrong to think of $Z$ as a DTFT in the typical sense, but we'll defer those details to EECS 225A. Our analysis below still gets at the heart of the issue and the tradeoffs involved.

<!-- BEGIN QUESTION -->

**Q:** Suppose $\sigma = 0$. Can we perfectly recover $X$, assuming that $|H(\omega)| > 0\ \forall \omega$? Explain.

**Q:** As $\sigma$ increases, intuitively, is our noise getting stronger or weaker? Is it easier or more difficult to recover $X$?

**Q:** The impulse response of our degradation system, which caused the echo, is $h(n) = \delta(n) + \alpha \delta(n -k)$, and so its frequency response is $H(\omega) = 1 + \alpha e^{-i\omega k}$. For concreteness, take $k=1$ and $\alpha=.8$. 

Which range of frequencies do you expect the noise amplification to be worst at: high ones, or low ones? Why? (*Hint*: Compute $|H(0)|$ and $|H(\pi)|$).

<span style="color:blue">**A:** (TODO) </span>
    
<span style="color:blue">**A:** (TODO) </span>
    
<span style="color:blue">**A:** (TODO) </span>

<!-- END QUESTION -->

# Q2: *Picture* This - 2D Signal Processing!
Nearly all the techniques covered thus far in the class and this lab for 1 dimensional signals are only a few modifications away from being applied to multi-dimensional ones. In this question we'll delve into the math and mystery of operating with 2 dimensional signals called images!

## Q2a: Intro to Image Processing

### Images as Signals
Images are just ordinary discrete-time signals that like to disguise themselves as rectangles, here's how:  

Imagine we have a discrete-time signal of duration $N$, and our desired image is a matrix of width $W$ and height $H$. We assert that $N=W\cdot H$, so that each sample from the signal has a spot in the image matrix. We can now, by a convention of the imaging world, start building our image by taking the first $W$ elements of our 1D signal and inserting them as the top row of the image matrix, the next $W$ elements of our signal become the second row of the matrix, and so on until we've ran out of both samples and space in the matrix; the process visually looks something like this:

<img src="figs/imagematrix.png" alt="drawing" width="1000px"/>

Thus an image is just a matrix, and that matrix is just a folded up discrete-time signal. In the context of images, the signal is typically a measure of **luminance**, or the amount of light present at a given 2D spatial coordinate. But how do we go from a matrix of numbers to a picture of a flower?  

There are many ways of encoding images, but for this lab we'll pretend that it's still 1850 and colour photography hasn't been invented. Our images are what's called "grayscale" or "uint8" images; a single matrix with values in the 8 bit range of $2^0-1=0$ (pure black) to $2^8-1=255$ (pure white). Values between these two correspond to various shades of gray.

Now let's load an image with `plt.imread` and get some information about it with `np.info`:

In [ ]:
flower1 = plt.imread("images/flower1.tiff")  # load the image file
print(np.info(flower1)) # return info on the file

If we look at the shape and type rows we can see that the image really is just a 256 x 256 numpy array of 8 bit unsigned integers (i.e. only non-negative values allowed), referred to by the type `uint8`. You can ignore the other data fields for now.

Now, run the cell below to see an example of displaying an image.

In [ ]:
plt.figure(figsize=(6, 6))

# for plt.imshow remember to set vmin=0, vmax=255 so it doesn't try to normalize the image
plt.imshow(flower1, cmap="gray", vmin=0, vmax=255) # display image data. cmap = colour map
plt.ylabel("y")
plt.xlabel("x")
plt.show()

Using matplotlib's imshow function we are able to see that it is an image of a flower, but what about the underlying signal, what does it look like?

In [ ]:
row = 128

plt.figure(figsize=(16,2))
plt.imshow(flower1[row][None,:], aspect='auto', cmap="gray")
plt.title("Row {0} of Flower Image".format(row))
plt.xlabel("Pixel Index along x-axis")
plt.show()

plt.figure(figsize=(16,4))
plt.plot(flower1[row]) # index into 2D array, return specified row
plt.xlabel("Pixel Index along x-axis")
plt.ylabel("Pixel Luminance (arbitrary units)")
plt.title("Luminance of Row {0} of Flower Image".format(row))
plt.xlim([0, 255])
plt.show()

If we take a look at the 128th row of the image, or equivalently the 128th chunk of the signal, we can see two peaks in luminance (brightness) centered around x=50 and x=200, as well as a trough at x=125. Looking back at the image we see that if we took such a slice through the middle of the image, we'd see two patches of bright petals and a patch of dark pistil (center of the flower) matching these peaks and trough.  

Feel free to change the row variable and explore the shapes of signal chunks produced by other slices of the flower.

### Root Mean Square Error (RMSE)

So now we can load and understand images visually, but what we really need is a good way to compare them numerically.  
We'll use the RMSE, or [Root Mean Square Error](https://en.wikipedia.org/wiki/Root-mean-square_deviation), a popular image processing similarity/error metric:

$$RMSE = \sqrt{\frac{1}{N}\sum_{n=0}^{N-1}(x_{1}[n]-x_{2}[n])^{2}}=\sqrt{\frac{1}{W\cdot H}\sum_{x=0}^{W-1}\sum_{y=0}^{H-1}(f_{1}[y,x]-f_{2}[y,x])^{2}}$$

Where $y_1, y_2$ are the image signals and $N$ is their size, or alternatively $f_1,f_2$ are image matrices with heights $H$ and widths $W$. This error formula penalizes large differences in luminance very heavily (by squaring them!). To relate this back to what you've already seen, this is the 2D version of taking the energy of the (now 2D) error signal $y_1[n] - y_2[n]$, but then taking the square root of that energy and normalizing it by multiplication with $1 / \sqrt{N}$.

Implement the RMSE function below.

In [ ]:
def RMSE(im1, im2):
    """
    Returns the Root Mean Square Error between two images.
    Parameters:
    im1     - Image 1, 2D numpy array
    im2     - Image 2, 2D numpy array
    """
    assert im1.shape == im2.shape, "shape mismatch"
    im1, im2 = im1.astype(np.float64), im2.astype(np.float64) # avoid overflow errors
    ...

In [ ]:
grader.check("q2a")

Lets check the RMSE of the flower and the lizard.

As a sanity check on your answers, consider the following questions (you don't need to answer them here, but they're worth considering):  
What is the maximum and minimum RMSE we can have if our data is limited to the range 0-255? What do we expect to be the RMSE between an image and itself?

In [ ]:
flower1 = plt.imread("images/flower1.tiff")
lizard = plt.imread("images/lizard.tiff")
print(RMSE(flower1,lizard))
print(RMSE(flower1, flower1))

Now that we can understand and compare images we're all set to process them!

## Q2b: *2D* Convolution? 1D Was Hard Enough Already!
2D convolution is conceptually similar to 1D convolution, but there's a trick. If we slid a 1D filter over the entire image signal row by row, we would only being able to filter features along the x axis, and would carry over information from one edge of the image to the other. Thus our filter has to also be 2D, often called a "kernel", and flipped/slid both along the x and y axes.

The process of 2D convolution should seem familiar: you first flip the kernel along the x and y axes, and then slide it along the image matrix. The kernel will always be of an odd dimension, typically 3x3 or 5x5, and thus have exactly one center point. This center point is aligned to a pixel in your image matrix, and the output pixel is the dot product of the kernel at that position and that portion of the image matrix.  

If the kernel "goes over the edge" of the image matrix we can deal with it just as in the 1D case, by either pretending the image matrix is padded by zeroes (what we will typically do), or wrap the kernel around to the other side of the matrix (helps to combat some edge artifacts). The second full method is demonstrated in the helpful .gif below (run the cell to see it):

In [ ]:
from IPython.display import HTML
HTML('<img src="https://upload.wikimedia.org/wikipedia/commons/1/19/2D_Convolution_Animation.gif">')

### Image Interpretation of Frequencies

2D kernel convolution has entire chapters of textbooks dedicated to its intricacies (A good reference text is Digital Image Processing by Gonzalez and Woods), and so the main take-away for now is that it's just sliding and dot products as in the 1D case.

Kernels are filters and so can be classified as low-pass, high-pass, band-pass and band-stop just like in 1D. But what exactly does a low-pass filter mean for an image? Low-pass filtering music will leave you with just the sound of the bass, but in 2D it'll leave you with a blurrier image:

<img src="figs/frequency.png" alt="drawing" width="800px" align="center"/>  

For images, signal elements represent luminance, so high frequencies correspond to rapid changes in this luminance, like what you'd find in a checkerboard pattern. On the opposite end of the spectrum (pun intended), lower frequencies represent smoother visual transitions like the slow gradient of a sunset.

### Gaussian Blur

Let's try out what we've learned so far by convolving an image matrix with a gaussian kernel, which will take a weighted average of patches of the image, hopefully acting like a low-pass filter. The gaussian kernel is given by:

$$\text{gkern}(x,y) = \dfrac{1}{2\pi\sigma^{2}}\exp\left\{ \dfrac{-(x^{2}+y^{2})}{2\sigma^{2}}\right\} $$

The effect of convolving an image with this filter is often referred to as "Gaussian blur", due to the fact that 2D low-pass filtering is just blurring, as discussed above. If you go into image editing software such as Adobe Photoshop, you'll find commands that allow you to apply this same Gaussian blur to images - this filtering operation is what Photoshop is doing behind the scenes!

In [ ]:
def gkern(size=5, sigma=1.0):
    """
    Returns a gaussian kernel with zero mean.
    
    Adapted from:
    https://stackoverflow.com/questions/29731726/how-to-calculate-a-gaussian-kernel-matrix-efficiently-in-numpy
    
    Parameters:
    size    - Sidelength of gaussian kernel
    sigma   - Value of standard deviation
    """
    ax = np.arange(-size // 2 + 1.0, size // 2 + 1.0)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2.0 * sigma**2))
    return kernel / np.sum(kernel)

Code for producing the kernel is given above; it's up to you to use the [scipy.signal.convolve2d()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html) function to apply the filter to the **lizard.tiff** image.  

**IMPORTANT: Use `mode="same"` and keep `boundary` as the default value (padding the image matrix with zeroes before convolution).**

We'll experiment with different values of "sigma" and "size" and be prepared to report on their effects. Your task is to use plt.imshow() to display 4 images: the original lizard, plus three **labelled** blurrings of it. So in total, you must create and plot:
1. The original lizard image.
2. A version blurred with **high sigma and small kernel size**.
3. A version blurred with **low sigma and large kernel size**.
4. A version blurred with **high sigma and large kernel size**.

Definitions of "high/low sigma" and "high/low kernel size" have been given for you in the cell.

How you actually display the images (e.g., in a 2x2 grid using `plt.subplot`, or just by making 4 separate figures) is up to you; we don't care for grading purposes.  

**Don't forget to set `cmap="gray"` in your calls to `plt.imshow`**

<!-- BEGIN QUESTION -->



In [ ]:
high_sigma = 5
low_sigma = 1
big_kernel = 50
small_kernel = 5

...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Q: What visual effect does increasing sigma have for blurring? What happens to the gaussian kernel (what other filter does it approach)?**

**Q: What visual effect does increasing kernel size have for blurring? Now change mode="same" to mode="full" in all the signal.convolve2d calls in the cell above and run it again. What happens to the edges of the image (do they get brighter/darker?), why?**

**Q: What are the first visual features of the lizard to be blurred away? What are the last? What does this say about the frequencies of those features?**


<span style="color:blue">A: (TODO) </span>

<span style="color:blue">A: (TODO) </span>

<span style="color:blue">A: (TODO) </span>

<!-- END QUESTION -->

## Q2c: Image Sharpening

Another feature in image processing software is the "sharpening" function. Here, we'll see how we can sharpen a picture by blurring it! If we blur an image, and subtract this blur from the original image, we end up erasing a lot of its low-frequency components and make it look comparatively sharper.

1. Try sharpening **flower2.tiff**, experiment with different values of sigma, kernel size, and alpha/beta (to perform a weighted subtraction of the image and it's blur) and use plt.imshow() to display the original picture and its sharpened version.

You should definitely be able to see some sharpening, especially around the edges of the leaves, but the final result won't  be amazing. In Question 2c, we'll try to find a smarter way of approaching this general problem of image denoising.

<!-- BEGIN QUESTION -->



In [ ]:
alpha = ...
beta = ...
flower2 = plt.imread("images/flower2.tiff")
gaussian_kernel = ...
flower2_blurred = ...
flower2_sharpened = alpha*flower2 - beta*flower2_blurred


# Plot
plt.figure(figsize=(20,10))
plt.subplot(1, 3, 1)
plt.imshow(flower2, cmap="gray", vmin=0, vmax=255)
plt.title("Unsharpened")

plt.subplot(1, 3, 2)
plt.imshow(flower2_blurred, cmap="gray", vmin=0, vmax=255)
plt.title("Blurred")

plt.subplot(1, 3, 3)
plt.imshow(flower2_sharpened, cmap="gray", vmin=0, vmax=255)
plt.title("Sharpened")

plt.show()

<!-- END QUESTION -->

## Q2D: 2D DFT

Finally, before discussing 2D deconvolution, let's reintroduce one last method from our 1D signal analysis toolbox: the Discrete Fourier Transform. The 2D DFT is an advanced topic which is explored in more detail in future classes such as EECS 225B, but for now let's just figure out how to read the magnitude plot of the 2D DFT. Just like we typically do for the 1D DFT, we will use `np.fft.fftshift` to center the lowest frequency (DC) in the middle of the plot. The 2D DFT formula looks exactly like the 1D formula, but summed over two axes:

$$ F(\omega_x,\omega_y)=\frac{1}{M\cdot N}\sum_{x=0}^{M-1}\sum_{y=0}^{N-1}f(y,x)e^{-2\pi\left(\dfrac{k}{M}x+\dfrac{l}{N}y\right)} $$

In the 2D DFT we thus have two directions in which we care about frequencies appearing in, x and y; and so we instead of choosing an N point DFT we now choose to sample a (M,N) point 2D DFT. In return we get a 2 dimensional magnitude plot which will show us the distribution of frequencies along both axes, as displayed below (after being fft-shifted):

<img src="figs/2d_dft.png" alt="drawing" width="800px"/>

As we can see, low frequencies are now in the center of the 2D plot, and high frequencies tend towards the corners.

Now a demonstration of calculating and plotting the 2D DFT of a test image:

In [ ]:
test_stripes = plt.imread("images/test_stripe.tiff")
plt.figure(figsize=(8, 8))
plt.imshow(test_stripes, cmap="gray")
plt.title("Horizontal Stripe")
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
test_fft = np.fft.fft2(test_stripes) # take 2D DFT with default parameters
test_fft = np.fft.fftshift(test_fft) # shift low frequencies to center of plot
test_fft = np.abs(test_fft) # calculate magnitude of DFT
plt.imshow(test_fft)
plt.colorbar()
plt.title("2D DFT of horizontal stripe")
plt.show()

This plot is a little hard to see, so let's zoom into the middle:

In [ ]:
plt.figure(figsize=(8, 8))
test_fft = np.fft.fft2(test_stripes) # take 2D DFT with default parameters
test_fft = np.fft.fftshift(test_fft) # shift low frequencies to center of plot
test_fft = np.abs(test_fft) # calculate magnitude of DFT
plt.imshow(test_fft, aspect="auto")
plt.xlim(231,235)
plt.colorbar()
plt.title("2D DFT of horizontal stripe")
plt.show()

Looking at this magnitude plot for the 2D DFT of horizontal stripes, we see that its energy is concentrated at x=233, the middle of the plot, and radiates up and down. Looking at our 2D DFT chart above, we see that this means that we have mostly low frequencies in the y direction, and have nothing interesting happening in the x direction of the image. This should make sense given that our image is completely static along the x axis, each column being a copy of the previous; no change = DC = zero extent in the 2D DFT.

Now let's take a vertical slice of the image and the 2D DFT:

In [ ]:
plt.figure(figsize=(16, 4))
plt.subplot(1, 2, 1)
plt.plot(test_stripes[:,233])
plt.xlim([0, 511])
plt.title("Column of Horizontal Stripes")

plt.subplot(1, 2, 2)
plt.plot(test_fft[:,233])
plt.title("Column of 2D DFT")
plt.show()

The vertical slice of the image looks like a rectangle (a 'rect' signal), whose fourier transform we know is a sinc function. And that's exactly what the vertical slice of the 2D DFT appears to be!

This is not a coincidence, and exploration of this relationship between image slices and 2D DFT slices led to the discovery of the [Radon transform](https://en.wikipedia.org/wiki/Radon_transform), which in turn led to the development of medical technologies such as [Computed Tomography Scanning](https://en.wikipedia.org/wiki/CT_scan).

If we look at the 2D DFT of a 'natural' image, like flower1, you'll often only see a dot:

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(np.abs(np.fft.fftshift(np.fft.fft2(flower1))))
plt.title("2D DFT log-Magnitude of flower image")
plt.colorbar()
plt.show()

This is because natural scenery rarely contains much high-frequency content. Apart from their edges, object luminances tend to change smoothly rather than looking like TV static. Natural images are thus almost entirely low-frequency content, meaning almost all energy of the 2D DFT would be concentrated in its center. You can read more about this phenomenon in  [**this paper**](http://web.mit.edu/torralba/www/ne3302.pdf).

It's often more interesting to look at the **log**-magnitude of a 2D DFT (essentially changing the magnitude scale to decibels, just as you saw for bode plots in 16B).

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(np.log(abs(np.fft.fftshift(np.fft.fft2(flower1))))) # take the log!
plt.title("2D DFT log-Magnitude of flower image")
plt.colorbar()
plt.show()

Now it's your turn: Plot the 2D DFT **log**-magnitude of the lizard image from before and the 2D DFT magnitude of its blurriest variant (the high sigma and high kernel size one), and see if you can spot what happened to the distribution of frequencies in the image.

<!-- BEGIN QUESTION -->



In [ ]:
...

**Q: Where in the 2D DFT magnitude plot is most of the energy concentrated for the blurred image? What frequencies (and in what directions) does this correspond to?**

_Type your answer here, replacing this text._

<!-- END QUESTION -->

# Q3: Image Deconvolution, The 1.5 Billion Dollar Problem

Now that we've gained some intuition about image matrix manipulation, convolution, and the 2D DFT, we can tackle a real-life problem which plagued scientists working on the **Hubble Space Telescope** in the 90's. The problem was that light, like most things in the universe, likes to spread out over distance. If you've ever pointed a flashlight into the night sky:  

<br>
<center> <img src="figs/light.jpg" alt="drawing" width="400px"/></center>
<br>

You've probably noticed that light spreads out in a cone shape rather than stay a cylindrical beam. This light spread is reasonably modeled via a Gaussian distribution, as are most things in life. The tricky bit comes down to figuring out what exact parameters — sigma and kernel size — to use. 

## The Hubble Space Telescope Disaster

When scientists first received images from the Hubble Space Telescope, they were devastated to find them all catastrophically blurred! As it turned out, the main mirror of the telescope had been polished into the [wrong shape](https://en.wikipedia.org/wiki/Hubble_Space_Telescope#Flawed_mirror), meaning the light hitting the sensor was out of focus, and exhibited this same Gaussian spread as we discussed in the flashlight example.  

The engineers on the Hubble team were not about to let a 1.5 billion dollar project go to waste for one faulty mirror, and so they turned to math for the answer, holding a workshop with the world's leading experts on image and signal processing present to try and tackle the problem. They ended up producing a 150 page manuscript of [solutions](http://www.stsci.edu/files/live/sites/www/files/home/hst/documentation/_documents/RestorationofHSTImagesandSpectra.pdf) to the issue. Here we'll go through one of the most successful ones, and the one that the Hubble team actually ended up using to fix the problem: **image deconvolution**. 

## Q3a: 2D Fourier Deconvolution

Just like the 1D example at the start of this lab, if we assume that our blur is modeled by a convolution of the original sharp image with a gaussian kernel, then for the 2D case

$$\hat{f}(x,y) = (f * g)(x,y) \implies \hat{F}(\omega_x,\omega_y) = F(\omega_x,\omega_y)\cdot G(\omega_x,\omega_y) \implies F(\omega_x,\omega_y) = \frac{\hat{F}(\omega_x,\omega_y)}{G(\omega_x,\omega_y)},$$

where $f(x,y)$ is the original image, $g(x,y)$ is the gaussian kernel, and $\hat{f}(x,y)$ is the resultant blurred image.    
  
Let's start by modifying the deconvolve function from earlier for 2D. Implement `deconvolve2d` below. The functions `np.fft.fft2` and `np.fft.ifft2` for handling 2D DFTs and inverse DFTs will be of use. Don't forget to handle numerical noise with `np.real` as before!   
  
**Hint:** Just like in 1D decovolution, make sure that the two FFT's are the same size when dividing them; you can use the size of the space images, `(512, 512)`, to set the size parameter in the np.fft functions, and they'll automatically zero-pad for you.

In [ ]:
def deconvolve2d(f_hat, g):
    """
    Perform a Fourier deconvolution to deconvolve g "out of" f_hat, assuming
    that g, f_hat and the resultant deconvolved image are purely real.
    
    Parameters:
    f_hat        - Image for deconvolution
    g            - Convolution kernel 
    """
    ...

In [ ]:
grader.check("q3a")

#### Sanity Check:
Make sure the code below runs without error, and that the image on the original left visually matches the deconvolved image on the right. This demonstrates the power of deconvolution if we know exactly what the gaussian kernel parameters are.

In [ ]:
test_kernel = gkern(20,20)
test_blurred = scipy.signal.convolve2d(flower2, test_kernel, mode="same", boundary='wrap')
test_deconvolved = deconvolve2d(test_blurred, test_kernel)

plt.figure(figsize=(15, 8))
plt.subplot(1, 3, 1)
plt.imshow(flower2, cmap="gray")
plt.title("Original Flower Image")

plt.subplot(1, 3, 2)
plt.imshow(test_blurred, cmap="gray")
plt.title("Blurred Flower Image")

plt.subplot(1, 3, 3)
plt.imshow(test_deconvolved, cmap="gray")
plt.title("Deconvolved Flower Image")
plt.show()

## Q3b: Playing NASA Engineer

It's 1991, and you're an image processing engineer at NASA. Your team has been working tirelessly to correct the issues with the Hubble Space Telescope. A mission to install corrective optics is being planned but won't happen for another two years; in the meantime, you (and your government funding sources) don't want all of the pictures taken to go to waste. Fortunately, your colleagues have made a reconstruction of **space1_blurred.npy**, manually creating a sharp **space1.tiff** to use as a model dataset. Running the cell below will display the telescope's original captured image and the manually sharpened one.

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(np.load("space1_blurred.npy"), cmap="gray")
plt.title("Original Hubble Image")

plt.subplot(1, 2, 2)
plt.imshow(plt.imread("images/space1.tiff"), cmap="gray")
plt.title("Manually Reconstructed Image")
plt.show()

The scientists you work with were happy to provide you with this test data, but what they really want is an automated way to perform this type of enhancement using computer software. You've chosen to model the mirror's defects as a Gaussian blur, and you know that this is an LTI operation, and so you can undo it via deconvolution! 

But how do we find the parameters for the Gaussian?

## Brute Force Optimization: The Parametric Sweep

If our search space is small and our problem is hard to optimize analytically, we can use a *parametric sweep*, very common in any field of engineering. For example, in machine learning, this is often called a *grid search*. In a parametric sweep, we iterate over all possible combinations of the parameters we want to tune — here, just kernel size and sigma — and return the ones that give us the smallest error. In this case we want the lowest RMSE between the result of the deconvolution and the scientists' manual reconstruction.

To implement the parameteric sweep: 
1. Iterate through all combinations of $(\text{Kernel Size}, \sigma)$ for $\text{Kernel Size} \in \{1, 3, 5, 7, ..., 19, 21, 23, 25\}$ (remember, kernel size must be odd) and $\sigma \in [0, 10]$. Since you can't actually test every single real number in the interval of 0 to 10 for $\sigma$, just pick 50 values from that interval that are spaced out evenly. The function `np.linspace` will be useful.
    1. Using each pair $(\text{Kernel Size}, \sigma)$, apply the 2D Fourier deconvolution algorithm you implemented with `deconvolve2d` using a Gaussian kernel (`gkern`) on the blurred image.
    3. Compute the RMSE between the deconvolved result `space1_blurred` and the manually reconstructed image `space1_ideal`. 
    4. Save the RMSE, sigma, kernel size, and the gaussian kernel itself if this is the smallest RMSE we've calculated so-far.
2. Run the code cell below that will display the end results (in terms of both RMSE and the actual image) for you.

**Hint**: The standard way to track the "best result" for these types of situations is to declare some variable, say, `best_RMSE` before you iterate, and set it to some ridiculously high value (you can use `np.inf`, numpy's value representing infinity, if you'd like, but anything above 1000 is more than big enough). Here, you'll also need a `best_sigma`, `best_kernel_size`, and `best_kernel`. Each time you get a better (lower) RMSE, update these variables. While it's not required, printing out each of them every time they're updated would be very instructive in seeing how this optimization technique converges to the optimal pair $(\text{Kernel Size}, \sigma)$.

**Ignore any division by zero warnings; this is from our deconvolution function because we're dividing by very small values.**

<!-- BEGIN QUESTION -->



In [ ]:
# Load images
space1_blurred = np.load("space1_blurred.npy") # blurred image
space1_ideal   = plt.imread("images/space1.tiff")     # manually reconstructed image

best_sigma = None
best_kernel_size = None
best_kernel = None

...

In [ ]:
# Display results  
meas_RMSE = RMSE(space1_blurred, space1_ideal)
print("Original RMSE: " + str(round(meas_RMSE, 3)))
print("Best RMSE:     " + str(round(best_RMSE, 3)))
ratio = meas_RMSE / best_RMSE
print("Numerically optimized Fourier deconvolution is {0} better in terms of RMSE.".format(round(ratio, 3)))

plt.figure(figsize=(15, 8))
plt.subplot(1, 3, 1)
plt.imshow(space1_blurred, cmap="gray")
plt.title("Original Space Hubble Image")

plt.subplot(1, 3, 2)
plt.imshow(deconvolve2d(space1_blurred, best_kernel), cmap="gray")
plt.title("Deconvolved Space Hubble Image")

plt.subplot(1, 3, 3)
plt.imshow(space1_ideal, cmap="gray")
plt.title("Manually Reconstructed Hubble Image")
plt.show()

<!-- END QUESTION -->

Results looking good? Nice!

Now, since we said that the corruption with our space telescope is consistently a Gaussian blur, and consistently the same kernel (since it's ultimately the telescope itself rather than some other random phenomenon), applying this same deconvolution method with the same kernel to any image it captures should give us the same kind of improvement! 

Let's use our calculated best gaussian kernel, `best_kernel`, to attempt to deconvolve another image of space that was captured, **space2_blurred.npy**, without having to manually reconstruct it.

To complete the lab, run the cell below (no code on your part required), which will apply the same deconvolution technique to this newly acquired image and display the result. After running it, you're all done.

In [ ]:
space2_blurred = np.load("space2_blurred.npy")
plt.figure(figsize=(15, 8))
plt.subplot(1, 2, 1)
plt.imshow(space2_blurred, cmap="gray")
plt.title("Blurred Hubble Image #2")

plt.subplot(1, 2, 2)
plt.imshow(deconvolve2d(space2_blurred, best_kernel), cmap="gray")
plt.title("Deconvolved Hubble Image #2")
plt.show()

**A parting note on the instability of 2D deconvolution:** 2D deconvolution is such an [ill-conditioned](https://en.wikipedia.org/wiki/Condition_number) problem that we had to give you numpy arrays for the blurred space images. This is because if we rounded the floating point data to its nearest integer representation, and saved it as a .tiff image file, that change would alone be enough that our deconvolution algorithm would never converge on an optimal solution (the error would always be massive, even for the "correct" gaussian kernel parameters).

# References

[1] *Signal reflection (Wikipedia)*. [Link](https://en.wikipedia.org/wiki/Signal_reflection)  
[2] *AT&T Archives: Similarities of Wave Behavior*. [Link](https://www.youtube.com/watch?v=DovunOxlY1k)  
[3] *Audio feedback (Wikipedia)*. [Link](https://en.wikipedia.org/wiki/Audio_feedback)  
[4] *Dereverberation (Wikipedia)*. [Link](https://en.wikipedia.org/wiki/Dereverberation)  
[5] *Stereophonic Acoustic Echo Cancellation: Theory and Implementation*. [Link](http://lup.lub.lu.se/search/ws/files/4596819/1001945.pdf)  
[6] *Restoration of Hubble Space Telescope Images and Spectra*. [Link](http://www.stsci.edu/hst/HST_overview/documents/RestorationofHSTImagesandSpectra.pdf)  
[7] *Richardson-Lucy deconvolution (Wikipedia)*. [Link](https://en.wikipedia.org/wiki/Richardson%E2%80%93Lucy_deconvolution)  
[8] *Wiener deconvolution (Wikipedia)*. [Link](https://en.wikipedia.org/wiki/Wiener_deconvolution)  
[9] *Signals, Systems, and Inference, Chapter 11: Wiener Filtering*. [Link](https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-011-introduction-to-communication-control-and-signal-processing-spring-2010/readings/MIT6_011S10_chap11.pdf)  
[10] 2D convolution GIF. [Link](https://upload.wikimedia.org/wikipedia/commons/4/4f/3D_Convolution_Animation.gif)  
[11] *LTI Models and Convolution, Section 11.2.3: Deconvolution*. [Link](https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-02-introduction-to-eecs-ii-digital-communication-systems-fall-2012/readings/MIT6_02F12_chap11.pdf)  
[12] *The Scientist and Engineer's Guide to Digital Signal Processing: Chapter 17, Custom Filters and Deconvolution*. [Link](https://www.dspguide.com/ch17/2.htm)  
[13] *Statistics of natural image categories*. [Link](http://web.mit.edu/torralba/www/ne3302.pdf)

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

Please upload the zip file produced by the result of this command to Gradescope.

In [ ]:
grader.export(force_save=True, run_tests=True)